In [1]:
import tensorflow as tf

# モデルの読み込み
model = tf.keras.models.load_model('c1.h5')

# モデルの入力形状を取得
input_shape = model.input_shape  # 通常、(None, height, width, channels)の形式

# TensorFlowはNHWC形式を使用するため、BCHW形式に変換
B, H, W, C = input_shape
print(f"BCHW format: (Batch size: {B}, Channels: {C}, Height: {H}, Width: {W})")

# 入力名と出力名を取得
input_names = [input_layer.name for input_layer in model.inputs]
output_names = [output_layer.name for output_layer in model.outputs]

print(f"Input Names: {input_names}")
print(f"Output Names: {output_names}")

# SavedModel形式としてエクスポート
model.save('saved_model_directory')

BCHW format: (Batch size: None, Channels: 3, Height: 24, Width: 320)
Input Names: ['img_in']
Output Names: ['n_outputs0/BiasAdd:0', 'n_outputs1/BiasAdd:0']
INFO:tensorflow:Assets written to: saved_model_directory/assets


In [2]:
!python -m tf2onnx.convert \
    --saved-model saved_model_directory \
    --output model.onnx \
    --opset 13 \
    --inputs 'img_in:0' \
    --inputs-as-nchw 'img_in:0' \
    --target tensorrt

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2024-02-12 18:46:20,500 - WARNING - ***IMPORTANT*** Installed protobuf is not cpp accelerated. Conversion will be extremely slow. See https://github.com/onnx/tensorflow-onnx/issues/1557
2024-02-12 18:46:22,984 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2024-02-12 18:46:32,707 - INFO - Signatures found in model: [serving_default].
2024-02-12 18:46:32,707 - WARNING - '--signature_def' not specified, using first signature: serving_default
2024-02-12 18:46:32,707 - INFO - Output names: ['n_outputs0', 'n_outputs1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2024-02-12 18:46:34,131 - WARNING - From /virtualenv/python3/lib/python3.6/site-packages/tf2onnx/tf_loader.py:715: extract_sub_graph (from tenso

In [10]:
import onnx
from onnx import numpy_helper

# モデルを読み込む
model_path = 'model.onnx'  # ONNXモデルファイルのパス
model = onnx.load(model_path)

# 入力層の情報を表示
print("Model's input details:")
for input in model.graph.input:
    # 入力層の名前
    name = input.name
    # データ型
    data_type = input.type.tensor_type.elem_type
    # シェイプ
    shape = [dim.dim_value for dim in input.type.tensor_type.shape.dim]
    print(f"Name: {name}, Type: {data_type}, Shape: {shape}")

# 出力層の情報を表示
print("\nModel's output details:")
for output in model.graph.output:
    # 出力層の名前
    name = output.name
    # データ型
    data_type = output.type.tensor_type.elem_type
    # シェイプ
    shape = [dim.dim_value for dim in output.type.tensor_type.shape.dim]
    print(f"Name: {name}, Type: {data_type}, Shape: {shape}")

Model's input details:
Name: img_in:0, Type: 1, Shape: [1, 3, 24, 320]

Model's output details:
Name: n_outputs0, Type: 1, Shape: [1, 1]
Name: n_outputs1, Type: 1, Shape: [1, 1]


In [9]:
import onnx
from onnx import helper

def update_batch_size(model_path, output_path, batch_size=1):
    # モデルを読み込む
    model = onnx.load(model_path)
    
    # 入力および出力のテンソルのシェイプを更新
    for tensor in model.graph.input:
        # バッチサイズを設定
        tensor.type.tensor_type.shape.dim[0].dim_value = batch_size

    for tensor in model.graph.output:
        # バッチサイズを設定
        tensor.type.tensor_type.shape.dim[0].dim_value = batch_size

    # モデルを保存
    onnx.save(model, output_path)
    print(f"Updated model saved to {output_path}")

# 元のモデルのパス
model_path = 'model.onnx'
# 更新されたモデルの出力パス
output_path = 'model.onnx'

# バッチサイズを1に更新
update_batch_size(model_path, output_path, batch_size=1)

Updated model saved to model.onnx


In [11]:
!/usr/src/tensorrt/bin/trtexec --onnx=model.onnx --saveEngine=model.engine

&&&& RUNNING TensorRT.trtexec [TensorRT v8201] # /usr/src/tensorrt/bin/trtexec --onnx=model.onnx --saveEngine=model.engine
[02/12/2024-18:51:55] [I] === Model Options ===
[02/12/2024-18:51:55] [I] Format: ONNX
[02/12/2024-18:51:55] [I] Model: model.onnx
[02/12/2024-18:51:55] [I] Output:
[02/12/2024-18:51:55] [I] === Build Options ===
[02/12/2024-18:51:55] [I] Max batch: explicit batch
[02/12/2024-18:51:55] [I] Workspace: 16 MiB
[02/12/2024-18:51:55] [I] minTiming: 1
[02/12/2024-18:51:55] [I] avgTiming: 8
[02/12/2024-18:51:55] [I] Precision: FP32
[02/12/2024-18:51:55] [I] Calibration: 
[02/12/2024-18:51:55] [I] Refit: Disabled
[02/12/2024-18:51:55] [I] Sparsity: Disabled
[02/12/2024-18:51:55] [I] Safe mode: Disabled
[02/12/2024-18:51:55] [I] DirectIO mode: Disabled
[02/12/2024-18:51:55] [I] Restricted mode: Disabled
[02/12/2024-18:51:55] [I] Save engine: model.engine
[02/12/2024-18:51:55] [I] Load engine: 
[02/12/2024-18:51:55] [I] Profiling verbosity: 0
[02/12/2024-18:51:55] [I] Tactic